# YouTube Comments via YouTube API

This notebook utilizes Google's YouTube Data API to generate a dataset of YouTube comments from Japanese language videos that are trending in the region. It takes up to 250 videos depending on how many videos in the most popular chart for the region are categorized as being in the language to increase chances that comments on the video are in Japanese. From each video, up to 100 comments are grabbed along with all their replies.

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build
from pathlib import Path
from time import sleep

In [ ]:
# Load in credentials from environment variables
load_dotenv()
API_KEY = os.getenv('API_KEY')

# Initialize API client
youtube = build(
    'youtube', 'v3', developerKey=API_KEY
)

## Retrieve comments from YouTube

commentThreads() with 'snippet,replies' pulls comments and all their replies from a specified videoId.
- order='relevance' gets comments with lots of likes, which are more likely to have replies

In [ ]:
def retrieve_comments(video_id, max_results=10):
    
    # Make request to API and save as a variable
    request = youtube.commentThreads().list(
        part='snippet,replies',
        maxResults=max_results,
        order='relevance',
        videoId=video_id
    )
    try:
        response = request.execute()
        
        return response
    except:
        return None

In [ ]:
# For testing
# test_response = retrieve_comments(video_id='4V0UAhe8o5c')
# test_response

## Writing comment data to a Pandas DF

Takes in a response from an API call and a dictionary containing information about the video. Returns a dataframe created from a list of dictionaries that contain:

* channel - the channel's title
* video_id - YouTube's unique identifier of the video
* category_id - YouTube id that classifies what the content of the video is about
* text - original text of the comment before any edits
* date_published - date when the comment was made

In [ ]:
def extract_info(response, video):
    
    comment_data = []
    
    for item in response['items']:
        
        # Grab the top-level comment first
        comment_data.append({
            'channel': video['channel'],
            'video_id': video['video_id'],
            'category_id': video['category_id'],
            'text': item['snippet']['topLevelComment']['snippet']['textOriginal'],
            'date_published': item['snippet']['topLevelComment']['snippet']['publishedAt']
        })
        
        # Check if there are replies and get same info if there are
        if 'replies' in item.keys():

            for reply in item['replies']['comments']:

                comment_data.append({
                    'channel': video['channel'],
                    'video_id': video['video_id'],
                    'category_id': video['category_id'],
                    'text': reply['snippet']['textOriginal'],
                    'date_published': reply['snippet']['publishedAt']
                })
        
    return pd.DataFrame(comment_data)

## Retrieve popular videos

videos().list() method with chart='mostPopular' generates a resource that has information about the currrently trending videos within a region.

There is a limit on the amount of videos displayed in the response but within the results is a token is given that can generate the next page of results. The function accepts this token as a parameter, which when combined with a loop allows for all the results to be retrieved up to the maximum specified.

In [ ]:
def retrieve_videos(next_page=''):
    request = youtube.videos().list(
        part='snippet',
        chart='mostPopular',
        maxResults=250,
        pageToken=next_page,
        regionCode='JP',
    )
    response = request.execute()
 
    return response

In [ ]:
def list_videos(response):
    
    for item in response['items']:
        
        snippet = item['snippet']
        
        if (snippet.get('defaultLanguage') == 'ja' or
            snippet.get('defaultAudioLanguage') == 'ja'):
      
            videos.append({
                'video_id': item['id'],
                'channel': snippet['channelTitle'],
                'category_id': snippet['categoryId']
    })

In [ ]:
# Make list
videos = []

# Run once to generate first page
response = retrieve_videos('')
next_page = response['nextPageToken']
list_videos(response)

# Loop until there is no nextPageToken
try:
    while next_page:
        response = retrieve_videos(next_page)
        next_page = response['nextPageToken']
        list_videos(response)
except:
    pass

In [ ]:
# Check list of videos
videos

In [ ]:
# Initialize a DataFrame to store data
comments_df = pd.DataFrame(columns=['channel',
                                    'video_id',
                                    'category_id',
                                    'text',
                                    'date_published'])

In [ ]:
# Loop through videos list       
for video in videos:
    
    # Query the YouTube Data API
    response = retrieve_comments(video['video_id'], max_results=100)
    sleep(3)

    # Add the data from the response to the DF
    if response:
        data = extract_info(response, video)
        comments_df = pd.concat([comments_df, data])

In [ ]:
# Check DF
comments_df

In [ ]:
# Export the comment data to a CSV file
output_path = Path('Resources/youtube_comments.csv')
comments_df.to_csv(output_path)